In [5]:
import cv2
import numpy as np
import os
import time
import scipy.io as sio
import matplotlib.pyplot as plt

import math
import main_functions as main

In [14]:
dl_dy= np.load("dl_dy.npy")
x=np.load("x.npy")
y= np.load("y.npy")
print(dl_dy.shape,x.shape,y.shape)

(7, 7, 3) (14, 14, 3) (7, 7, 3)


In [15]:
#dl_dy= np.load("dl_dy.npy")
#x=np.load("x.npy")
#y= np.load("y.npy")
dl_dy = np.array([[[1,2],[2,3]]]).reshape((2,2,1))
x = np.array([[[1,-1,0,2],[0.5,0,1,-4],[1,0,2,4],[1,2,3,-1]]]).reshape((4,4,1))
y = dl_dy+5

In [16]:
print(pool2x2_backward(dl_dy, x, y).reshape(4,4))
print(dl_dy.reshape(2,2))
print(x.reshape(4,4))

[[1. 0. 0. 2.]
 [0. 0. 0. 0.]
 [0. 0. 0. 3.]
 [0. 2. 0. 0.]]
[[1 2]
 [2 3]]
[[ 1.  -1.   0.   2. ]
 [ 0.5  0.   1.  -4. ]
 [ 1.   0.   2.   4. ]
 [ 1.   2.   3.  -1. ]]


In [17]:
def pool2x2_backward(dl_dy, x, y):
    # TO DO
    x_size = x.shape

    new_size = np.ceil(np.array(x_size)/2)

    dl_dx_pre = np.zeros((np.int(2*new_size[0]),np.int(2*new_size[1]),x_size[-1]))

    for k in range(x_size[-1]):

        K = 2
        L = 2
        add_1 = np.int(x_size[0]%K)
        add_2 = np.int(x_size[1]%L)

        x_padding  = np.pad( x[:,:,k], ((0,add_1), (0,add_2)), 'constant', constant_values=0)

        end_1 = np.int(new_size[0]*K)
        end_2 = np.int(new_size[1]*L)

        x_pre = x_padding[:end_1, :end_2].reshape(np.int(new_size[0]), K, np.int(new_size[1]), L)
        x_temp = np.zeros_like(x_pre)

        for i in range(np.int(new_size[0])):
            for j in range(np.int(new_size[1])):
                x_temp[i,:,j,:] = np.where( x_pre[i,:,j,:] == np.max( x_pre[i,:,j,:]), dl_dy[i,j,k] , 0)

        dl_dx_pre[:,:,k] = x_temp.reshape((np.int(2*new_size[0]),np.int(2*new_size[1])))

    
    #print(dl_dx_pre,add_1,add_2)
    if (add_1!=0):
        if (add_2!=0):
            dl_dx = dl_dx_pre[:-1, :-1, :]
        else:
            dl_dx = dl_dx_pre[:-1, :, :] 
    else:
        if (add_2!=0):
            dl_dx = dl_dx_pre[:, :-1, :]
        else:
            dl_dx = dl_dx_pre[:, :, :] 
        
    return dl_dx

In [18]:
def im2col_sliding_strided(A, BSZ, stepsize=1):
    # Parameters
    m,n = A.shape
    s0, s1 = A.strides
    nrows = m-BSZ[0]+1
    ncols = n-BSZ[1]+1
    shp = BSZ[0],BSZ[1],nrows,ncols
    strd = s0,s1,s0,s1

    out_view = np.lib.stride_tricks.as_strided(A, shape=shp, strides=strd)
    return out_view.reshape(BSZ[0]*BSZ[1],-1)[:,::stepsize]

In [53]:
def conv(x, w_conv, b_conv):
    # TO DO
    x_shape = x.shape
    w_conv_shape = w_conv.shape

    y = np.zeros((x_shape[0],x_shape[1],w_conv_shape[-1]))

    for id_i in range(x_shape[-1]):
        x_padding = np.pad(x[:,:,id_i], (1, 1), 'constant', constant_values=0)

        X = im2col_sliding_strided(x_padding, [w_conv_shape[0],w_conv_shape[1]])
        print(X)
        for id_j in range(w_conv_shape[-1]):
            w = np.flip(w_conv[:,:,id_i,id_j].reshape((w_conv_shape[0]*w_conv_shape[1])),axis=0)
            print(w)

            b = b_conv[:,id_j]

            #print(w.shape,X.shape,b_conv.shape)

            y[:,:,id_j] = y[:,:,id_j] +  (w @ X - b).reshape((x_shape[0],x_shape[1]))

    return y

In [56]:
x = np.array([[[1,2,3],[4,5,6],[7,8,9]]]).reshape((3,3,1))

w_conv = (np.array([[[-1,-2,-1],[0,0,0],[1,2,1]]])).reshape((3,3,1,1))

b_conv = np.array([[0]]).reshape((1,1))
conv(x, w_conv, b_conv).reshape((3,3))

[[0 0 0 0 1 2 0 4 5]
 [0 0 0 1 2 3 4 5 6]
 [0 0 0 2 3 0 5 6 0]
 [0 1 2 0 4 5 0 7 8]
 [1 2 3 4 5 6 7 8 9]
 [2 3 0 5 6 0 8 9 0]
 [0 4 5 0 7 8 0 0 0]
 [4 5 6 7 8 9 0 0 0]
 [5 6 0 8 9 0 0 0 0]]
[ 1  2  1  0  0  0 -1 -2 -1]


array([[-13., -20., -17.],
       [-18., -24., -18.],
       [ 13.,  20.,  17.]])

In [34]:
x = np.array([[[1,2,3],[2,3,4],[4,5,6]]]).reshape((3,3,1))
w_conv = (np.array([[[1,0,2],[0,-1,0],[3,0,4]]])).reshape((3,3,1,1))
b_conv = np.array([[0]]).reshape((1,1))

In [35]:
print(x.shape, w_conv.shape, b_conv.shape)
conv(x, w_conv, b_conv).reshape((3,3))

(3, 3, 1) (3, 3, 1, 1) (1, 1)


array([[11., 20.,  6.],
       [22., 40., 13.],
       [ 2.,  5., -3.]])

In [36]:
print(x.reshape((3,3)))
print(w_conv.reshape((3,3)))

[[1 2 3]
 [2 3 4]
 [4 5 6]]
[[ 1  0  2]
 [ 0 -1  0]
 [ 3  0  4]]


In [37]:
x = np.array([[[1,2,3],[4,5,6],[7,8,9]]]).reshape((3,3,1))
w_conv = (np.array([[[1,-2,1],[0,0,0],[1,2,1]]])).reshape((3,3,1,1))
b_conv = np.array([[0]]).reshape((1,1))

In [38]:
print(x.shape, w_conv.shape, b_conv.shape)
print(x.reshape((3,3)))
print(w_conv.reshape((3,3)))
conv(x, w_conv, b_conv).reshape((3,3))

(3, 3, 1) (3, 3, 1, 1) (1, 1)
[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[ 1 -2  1]
 [ 0  0  0]
 [ 1  2  1]]


array([[13., 20., 17.],
       [22., 32., 22.],
       [-3.,  0., -7.]])